# Projeto OCR com Notas Fiscais

## Indice

1. Introdução
2. Planejamento
3. Frameworks Utilizadas
4. Implementação
6. Conclusões

## Introdução

Este projeto é um protótipo de como retirar informações de notas fiscais por OCR e Expressões regulares. O objetivo foi ler as notas como uma imagem e extrair delas alguns pontos. As informações retiradas são:

1. Nome da empresa (LTDA)
2. CNPJ da empresa
3. Itens de compras
4. Total da compra

Claramente, há mais que isso numa nota fiscal, porém limitei nesses quatro pontos.

## Planejamento

Para a realização desse projeto foi feito seguinte processo:

1. Leitura da imagem e pre-processamento por filtros (Retirar coisas estranhas e planificar a imagem)
2. Leitura da imagem por um OCR treinado
3. Processar o texto 
4. Organizar o texto e coletar as informações

Houve um processo que foi omitido que falarei mais na conclusão.

## Frameworks utilizadas

Utilizei as seguintes frameworks:
- OpenCV 4.1 (pyOpenCV)
- pytesseract (Implementação do Google Tesseract para leitura de OCR)
- pandas (Tabela)

Além das bibliotecas do Python 3.7

## Implementação

In [9]:
import cv2
import pytesseract
import numpy as np
import pandas as pd
import re
import os

### 1.Leitura e pre-processamento de imagens

In [2]:
# Primeiramente, vamos carregar as fotos na memória. Para tal ele vai perguntar a direção da pasta dos cupons.
caminho = input("Qual a pasta dos cupons?\n")
arquivos_caminhos = os.listdir( caminho )

caminhos_imagens = []
#Aqui ele vai armazenar os paths para as imagens
for imagem_path in arquivos_caminhos:
    path = [caminho, '\\', imagem_path]
    caminhos_imagens.append("".join(path))

Qual a pasta dos cupons?
C:\Users\Pablo\Desktop\CPQi_Hackaton\Cupons


In [3]:
imagens_array = []

#Esse é o kernel uttilizado para a operação com os filtros, 
# nesse caso é um [5x5] para processar os ruídos na operação de Opening

kernel = np.ones((5,5),np.uint8)

# Percorrer as imagens para conseguir todas as informações

for caminho_imagem in caminhos_imagens:
    
    #Carrega a imagem original
    imagem_original = cv2.imread(caminho_imagem)
    
    #Coloca em preto e branco
    imagem_pb = cv2.cvtColor(imagem_original, cv2.COLOR_BGR2GRAY)
    
    #Usa um threshold para planificar sombras das imagens e tentar separar melhor as letras
    imagem_threshold = cv2.adaptiveThreshold(imagem_pb,255,
                                         cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
                                         cv2.THRESH_BINARY,11,5)
    
    #Usa um blur para diminuir o processo destrutivo das letras quando passam pelo o threshold
    imagem_blur = cv2.GaussianBlur(imagem_threshold,(11,11), 0)
    
    #Usa um filtro de opening para tentar equilibrar o blur com a definição do threshold para a leitura no OCR
    imagem_opening = cv2.morphologyEx(imagem_blur, cv2.MORPH_OPEN, kernel)
    
    #Armazena todas as imagens com os filtros para processamento
    imagens_array.append(imagem_opening)

### 2. Aplicando o OCR

In [4]:
#Armazena todos os textos das notas
texto_notas = []

i = 1

for imagem_nota in imagens_array:
    #Usa o processo do Google Tesseract para ler a imagem com OCR
    texto_notas.append(pytesseract.image_to_string(imagem_nota,lang="por"))
    
    #Mostra o progresso do processo
    print("Processou nota " + str(i) + "\n")
    i = i + 1

Processou nota 1

Processou nota 2

Processou nota 3

Processou nota 4

Processou nota 5

Processou nota 6

Processou nota 7



### 3.Processar o texto

In [5]:
strings_notas = []

#Percorre os textos de todas as notas e tira algumas leituras erradas
for texto_nota in texto_notas:
    
    #Junta todas as linhas dividas que são maior que três caracteres
    strings_notas.append([linha for linha in texto_nota.splitlines() if linha.strip() and len(linha)>3])

In [116]:
def encontra_nome_empresa (nota_strings):
    #Vai procurar no padrão de ser as duas primeiras linhas da nota fiscal o nome da empresa, caso encontra, devolve a str
    # caso não, devolve não encontrou
    for i in range (2):
        if nota_strings[i].find("LTDA") != -1:
            return nota_strings[i] 
    return ("Não encontrada")


def encontra_cnpj_empresa (nota_strings):
    for i in range (2,10):
        # O CNPJ está entre as primeiras linhas do CNPJ, porém variam muito em relação a posição, portanto, é preciso 
        # Procurar com o inicio em C ou c
        if nota_strings[i].find("C") == 0 or nota_strings[i].find("c") == 0:
            # Essa expressão regular significa pega a primeira parte sendo qualquer caractere, em segudo um dois pontos
            # Após isso pega a expressão do CNPJ, o resto liga com o IE.
            # match_cnpj = re.match(r'^(.+)(:)(.+)( )([A-Z]+)(.+)', nota_strings[i], re.I)
            # O terceiro grupo é o do CNPJ (Porém, falha no da freitas, assim foi deixado de lado)
            #cnpj = match_cnpj.group(3) 
            
            #Foi improvisado uma seleção, porém não é o ideal. 
            return i, nota_strings[i][4:24]
    #Devolve 0 para o indice de procura dos itens não ser prejudicada e pesquisar do inicio do arquivo, caso necessário
    return 0, ("Não encontrada")


def encontra_valor_total(nota_strings, linha_inicial):
    for i in range (len(nota_strings)):
        #Procura o valor total após a string do CNPJ pois esta no final da nota
        if nota_strings[i].find("TOTAL") != -1 and i > linha_inicial:
                #Acha com expressão regular o total do valor
                match_valor = re.match(r'(.+)( )(.+)', nota_strings[i], re.I)
                return i, match_valor.group(3)
    #Devolve o final do arquivo para o indice de procura dos itens não ser prejudicada e 
    # pesquisar do final do arquivo, caso necessário
    return len(nota_strings), ("Não encontrada")


def encontra_itens_nota(nota_strings, linha_inicial, linha_final):
    # Faz uma lista com todos os itens que podem ter numa nota
    lista_itens = []
    for i in range(linha_inicial, linha_final):
        match_item = re.match(r'^([0-9]{2,4})( )([0-9]+)( )(.+)', nota_strings[i], re.I)
        match_other_item = re.match(r'^([0-9]{2,10})( )(.+)', nota_strings[i], re.I)
        if match_item:
            lista_itens.append(match_item.group(5))
            continue

        if match_other_item:
            lista_itens.append(match_other_item.group(3))  
    return lista_itens

In [117]:
dados_notas = []

for nota in strings_notas:
    
    nome_empresa = encontra_nome_empresa(nota)
    indice_inicial_itens, cnpj_empresa = encontra_cnpj_empresa(nota)
    indice_final_itens, valor_nota = encontra_valor_total(nota, indice_inicial_itens)
    itens_nota = encontra_itens_nota(nota, indice_inicial_itens, indice_final_itens)
    dado_nota = [nome_empresa, cnpj_empresa, valor_nota, itens_nota]
    dados_notas.append(dado_nota)
    
print(dados_notas)

[['DOCEHANIA CHOCOLATES LTDA ME', ': 86.924.156/0001-84', '1070,20', ['417.063.0041905.32.004MINI LA', '417.002.0041806.31.104ALFAJOR', '417.003.0041806.32. 1021 ABLETE']], ['Não encontrada', ': 14,158.585/00 E: 0', '5.00', ['Casa Hista BEL 2unk 2.50(0.00)- 5.00']], ['COMERCIAL DE MIUDEZAS FREITAS LTDA', ' :63473235000978 1E:', '3,99', ['PORTA GARRAFA 004']], ['Não encontrada', '14,158.595/0002-02 T', 'Não encontrada', ['pata 1585 0600 0202 823 mo 9560 9797 9000 7760']], ['“COMERCIAL DE MIUDEZAS FREITAS LTDA.', ' “n3473235000976 Ar ', 'Não encontrada', []], ['Não encontrada', ': 14.158.585/0 Bi 16', '5.00', ['Casq Hista BL 2unk 2.500 0.00) «5.00']], ['“COMERCIAL DE MIUDEZAS FREITAS LTDA', ' :63473235000978 1E:', '3,99', ['PORTA GARRAFA 004']]]


### 4.4. Organizar o texto

In [118]:
pd.DataFrame(dados_notas)

,0,1,2,3
0,DOCEHANIA CHOCOLATES LTDA ME,: 86.924.156/0001-84,"1070,20","[417.063.0041905.32.004MINI LA, 417.002.004180..."
1,Não encontrada,": 14,158.585/00 E: 0",5.00,[Casa Hista BEL 2unk 2.50(0.00)- 5.00]
2,COMERCIAL DE MIUDEZAS FREITAS LTDA,:63473235000978 1E:,"3,99",[PORTA GARRAFA 004]
3,Não encontrada,"14,158.595/0002-02 T",Não encontrada,[pata 1585 0600 0202 823 mo 9560 9797 9000 7760]
4,“COMERCIAL DE MIUDEZAS FREITAS LTDA.,“n3473235000976 Ar,Não encontrada,[]
5,Não encontrada,: 14.158.585/0 Bi 16,5.00,[Casq Hista BL 2unk 2.500 0.00) «5.00]
6,“COMERCIAL DE MIUDEZAS FREITAS LTDA,:63473235000978 1E:,"3,99",[PORTA GARRAFA 004]


## Conclusão

Podemos notar certos problemas com a leitura dos cupons fiscais e também, com a estutura do Regex na parte de seleção.

Um problema, é que não houve um corte para somente ler o texto, isso se deve que a implementação do EAST estava problematica e não conseguia pegar o texto nas notas e a técnica de bleeding, que utiliza uma mascara para pegar o texto gerada por pegar as bordas e transformar em quadradados estava com uma demora bem considerável, porém é sim um ponto de melhora futuro para se fazer antes do OCR. Como exemplo, uma técnica de bleeding que foi implementada mas não colocada aqui

![Exemplo funcional da técnica de bleeding](OCR-filtro-futuro.png)

Outro problema, foi o Regex do CNPJ que funcionava para as notas, menos a da freitas varejo. Então, uam solução mais "Hard-coded foi feita, porém no caso do regex, fica assim.

In [121]:
def encontra_cnpj_empresa_regex (nota_strings):
    for i in range (2,10):
        # O CNPJ está entre as primeiras linhas do CNPJ, porém variam muito em relação a posição, portanto, é preciso 
        # Procurar com o inicio em C ou c
        if nota_strings[i].find("C") == 0 or nota_strings[i].find("c") == 0:
            # Essa expressão regular significa pega a primeira parte sendo qualquer caractere, em segudo um dois pontos
            # Após isso pega a expressão do CNPJ, o resto liga com o IE.
            match_cnpj = re.match(r'^(.+)(:)(.+)( )([A-Z]+)(.+)', nota_strings[i], re.I)
            # O terceiro grupo é o do CNPJ (Porém, falha no da freitas, assim foi deixado de lado)
            cnpj = match_cnpj.group(3) 
            
            #Foi improvisado uma seleção, porém não é o ideal. 
            return i, cnpj
    #Devolve 0 para o indice de procura dos itens não ser prejudicada e pesquisar do inicio do arquivo, caso necessário
    return 0, ("Não encontrada")

In [122]:
for nota in strings_notas:
    indice_inicial_itens, cnpj_empresa = encontra_cnpj_empresa_regex(nota)
    print(cnpj_empresa)

 86.924.156/0001-84
 14,158.585/00
000063016974
 14,158.595/0002-02
 “n3473235000976 Ar MONgo NBA
 14.158.585/0
000063016974


Por algum motivo,são colocados zeros que não estão na string original, que precisa de uma investigação maior.

Para melhorar o sistema, precisaria de:
- Melhorar o Regex da seleção de dados
- Fazer o passo intermediário de isolar o texto para leitura
- Testar melhor a acurrácia do sistema
- Pegar mais features que contém na nota, como se foi paga no Débito ou no Crédito

A leitura quando não falhava por ruído, era satisfatória, as leituras de preço foram corretas e as de CNPJ satisfatórias. Este é um simples protótipo que pode ser melhorado, foi feito para um projeto de teste de OCR e processamento de texto.